# การฝึก RL เพื่อควบคุมสมดุล Cartpole

สมุดบันทึกนี้เป็นส่วนหนึ่งของ [หลักสูตร AI สำหรับผู้เริ่มต้น](http://aka.ms/ai-beginners) ซึ่งได้รับแรงบันดาลใจจาก [คู่มือ PyTorch อย่างเป็นทางการ](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) และ [การใช้งาน Cartpole ด้วย PyTorch นี้](https://github.com/yc930401/Actor-Critic-pytorch)

ในตัวอย่างนี้ เราจะใช้ RL เพื่อฝึกโมเดลให้ควบคุมสมดุลของเสาที่ตั้งอยู่บนรถเข็น ซึ่งสามารถเคลื่อนที่ไปทางซ้ายและขวาบนแกนนอน เราจะใช้สภาพแวดล้อม [OpenAI Gym](https://www.gymlibrary.ml/) เพื่อจำลองเสาและรถเข็นนี้

> **Note**: คุณสามารถรันโค้ดของบทเรียนนี้ในเครื่องของคุณเอง (เช่น จาก Visual Studio Code) ซึ่งในกรณีนี้ การจำลองจะเปิดในหน้าต่างใหม่ หากคุณรันโค้ดออนไลน์ คุณอาจต้องปรับแต่งโค้ดเล็กน้อยตามที่อธิบายไว้ [ที่นี่](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7)

เราจะเริ่มต้นด้วยการตรวจสอบให้แน่ใจว่า Gym ได้รับการติดตั้งแล้ว:


In [ ]:
import sys
!{sys.executable} -m pip install gym

ตอนนี้เรามาสร้างสภาพแวดล้อม CartPole และดูวิธีการใช้งานกัน สภาพแวดล้อมมีคุณสมบัติดังนี้:

* **Action space** คือชุดของการกระทำที่เราสามารถทำได้ในแต่ละขั้นตอนของการจำลอง
* **Observation space** คือพื้นที่ของการสังเกตที่เราสามารถทำได้


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

มาดูกันว่าการจำลองสถานการณ์ทำงานอย่างไร ลูปด้านล่างนี้จะรันการจำลองจนกว่า `env.step` จะไม่คืนค่าสถานะสิ้นสุด `done` เราจะสุ่มเลือกการกระทำโดยใช้ `env.action_space.sample()` ซึ่งหมายความว่าการทดลองอาจล้มเหลวอย่างรวดเร็ว (สภาพแวดล้อม CartPole จะสิ้นสุดเมื่อความเร็ว ตำแหน่ง หรือมุมของ CartPole อยู่นอกขอบเขตที่กำหนด)

> การจำลองจะเปิดในหน้าต่างใหม่ คุณสามารถรันโค้ดหลายครั้งและดูว่ามันทำงานอย่างไร


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

คุณสามารถสังเกตได้ว่าการสังเกตการณ์ประกอบด้วยตัวเลข 4 ตัว ได้แก่:
- ตำแหน่งของรถเข็น
- ความเร็วของรถเข็น
- มุมของเสา
- อัตราการหมุนของเสา

`rew` คือรางวัลที่เราได้รับในแต่ละขั้นตอน ในสภาพแวดล้อมของ CartPole คุณจะได้รับรางวัล 1 คะแนนสำหรับแต่ละขั้นตอนการจำลอง และเป้าหมายคือการเพิ่มรางวัลรวมให้มากที่สุด นั่นคือเวลาที่ CartPole สามารถทรงตัวได้โดยไม่ล้ม

ในระหว่างการเรียนรู้แบบเสริมกำลัง เป้าหมายของเราคือการฝึก **นโยบาย** $\pi$ ซึ่งสำหรับแต่ละสถานะ $s$ จะบอกเราว่าควรดำเนินการ $a$ ใด ดังนั้นโดยพื้นฐานแล้ว $a = \pi(s)$

หากคุณต้องการวิธีแก้ปัญหาแบบมีความน่าจะเป็น คุณสามารถคิดว่านโยบายเป็นการคืนค่าความน่าจะเป็นสำหรับแต่ละการกระทำ เช่น $\pi(a|s)$ หมายถึงความน่าจะเป็นที่เราควรดำเนินการ $a$ ในสถานะ $s$

## วิธี Policy Gradient

ในอัลกอริทึม RL ที่ง่ายที่สุดที่เรียกว่า **Policy Gradient** เราจะฝึกเครือข่ายประสาทเทียมเพื่อทำนายการกระทำถัดไป


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

เราจะฝึกเครือข่ายโดยการรันการทดลองหลายครั้ง และอัปเดตเครือข่ายของเราหลังจากแต่ละครั้ง มากำหนดฟังก์ชันที่จะรันการทดลองและส่งคืนผลลัพธ์ (ที่เรียกว่า **trace**) - สถานะทั้งหมด, การกระทำ (และความน่าจะเป็นที่แนะนำของพวกมัน), และรางวัล:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

คุณสามารถรันหนึ่งตอนด้วยเครือข่ายที่ยังไม่ได้รับการฝึกฝนและสังเกตว่าผลตอบแทนรวม (หรือที่เรียกว่าความยาวของตอน) นั้นต่ำมาก:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

หนึ่งในแง่มุมที่ซับซ้อนของอัลกอริทึม policy gradient คือการใช้ **รางวัลที่ถูกลดทอน** แนวคิดคือเราคำนวณเวกเตอร์ของรางวัลรวมในแต่ละขั้นตอนของเกม และในระหว่างกระบวนการนี้เราลดทอนรางวัลในช่วงต้นโดยใช้สัมประสิทธิ์ $gamma$ นอกจากนี้เรายังปรับเวกเตอร์ที่ได้ให้เป็นมาตรฐาน เพราะเราจะใช้มันเป็นน้ำหนักเพื่อส่งผลต่อการฝึกอบรมของเรา:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

มาเริ่มการฝึกกันเลย! เราจะรันทั้งหมด 300 ตอน และในแต่ละตอนเราจะทำตามขั้นตอนดังนี้:

1. รันการทดลองและเก็บข้อมูลการทำงาน (trace)
2. คำนวณความแตกต่าง (`gradients`) ระหว่างการกระทำที่เกิดขึ้นจริงและความน่าจะเป็นที่คาดการณ์ไว้ ความแตกต่างยิ่งน้อยเท่าไร เราก็ยิ่งมั่นใจมากขึ้นว่าเราได้เลือกการกระทำที่ถูกต้อง
3. คำนวณรางวัลแบบลดค่า (discounted rewards) และคูณ `gradients` ด้วยรางวัลที่ลดค่าแล้ว - วิธีนี้จะช่วยให้ขั้นตอนที่มีรางวัลสูงส่งผลต่อผลลัพธ์สุดท้ายมากกว่าขั้นตอนที่มีรางวัลต่ำ
4. การกระทำเป้าหมายที่คาดหวังสำหรับ neural network ของเราจะมาจากความน่าจะเป็นที่คาดการณ์ไว้ระหว่างการรัน และบางส่วนมาจาก `gradients` ที่คำนวณได้ เราจะใช้พารามิเตอร์ `alpha` เพื่อกำหนดว่าส่วนของ `gradients` และรางวัลจะถูกนำมาพิจารณาในระดับใด - ซึ่งเรียกว่าค่า *learning rate* ของอัลกอริธึมการเสริมกำลัง
5. สุดท้าย เราจะฝึกเครือข่ายของเราด้วยสถานะและการกระทำที่คาดหวัง และทำซ้ำกระบวนการนี้


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

ตอนนี้มาลองรันตอนพร้อมการเรนเดอร์เพื่อดูผลลัพธ์:


In [ ]:
_ = run_episode(render=True)

หวังว่าคุณจะเห็นแล้วว่าเสาไม้สามารถทรงตัวได้ค่อนข้างดีเลยทีเดียว!

## โมเดล Actor-Critic

โมเดล Actor-Critic เป็นการพัฒนาต่อยอดจาก policy gradients โดยเราจะสร้างโครงข่ายประสาทเทียม (neural network) เพื่อเรียนรู้ทั้งนโยบาย (policy) และรางวัลที่คาดการณ์ไว้ (estimated rewards) โครงข่ายนี้จะมีสองผลลัพธ์ (หรือคุณอาจมองว่าเป็นสองโครงข่ายแยกกันก็ได้):
* **Actor** จะเสนอแนะการกระทำที่ควรทำ โดยให้การแจกแจงความน่าจะเป็นของสถานะ (state probability distribution) เช่นเดียวกับในโมเดล policy gradient
* **Critic** จะประเมินว่ารางวัลที่ได้จากการกระทำเหล่านั้นจะเป็นอย่างไร โดยจะคืนค่ารางวัลรวมที่คาดการณ์ไว้ในอนาคตสำหรับสถานะที่กำหนด

มาลองกำหนดโมเดลแบบนี้กัน:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

เราจำเป็นต้องปรับเปลี่ยนฟังก์ชัน `discounted_rewards` และ `run_episode` ของเราเล็กน้อย:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


ตอนนี้เราจะเริ่มการวนลูปการฝึกอบรมหลัก เราจะใช้กระบวนการฝึกอบรมเครือข่ายแบบแมนนวลโดยการคำนวณฟังก์ชันการสูญเสียที่เหมาะสมและอัปเดตพารามิเตอร์ของเครือข่าย:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## ข้อคิดสำคัญ

เราได้เห็นอัลกอริทึม RL สองแบบในเดโมนี้: simple policy gradient และ actor-critic ที่ซับซ้อนมากขึ้น คุณจะเห็นว่าอัลกอริทึมเหล่านี้ทำงานโดยใช้แนวคิดเชิงนามธรรมของสถานะ การกระทำ และรางวัล - ดังนั้นจึงสามารถนำไปใช้กับสภาพแวดล้อมที่แตกต่างกันได้อย่างมาก

การเรียนรู้แบบเสริมกำลังช่วยให้เราสามารถเรียนรู้กลยุทธ์ที่ดีที่สุดในการแก้ปัญหาได้เพียงแค่ดูที่รางวัลสุดท้าย ความจริงที่ว่าเราไม่จำเป็นต้องใช้ชุดข้อมูลที่มีการติดป้ายกำกับช่วยให้เราสามารถจำลองสถานการณ์ซ้ำหลายครั้งเพื่อปรับปรุงโมเดลของเราได้ อย่างไรก็ตาม RL ยังมีความท้าทายอีกมากมาย ซึ่งคุณอาจได้เรียนรู้หากคุณตัดสินใจที่จะมุ่งเน้นในด้านที่น่าสนใจนี้ของ AI



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์มืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
